### 쌍체 t 검정 수행

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
df = pd.read_csv('winequality-red.csv')

In [6]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from collections import Counter

# 예시로 사용할 데이터를 불러오는 코드
# 실제 데이터에 맞게 파일 경로나 데이터 로딩 방법을 수정해야 합니다.
df = pd.read_csv('winequality-red.csv')

# 'quality' 컬럼이 품질 클래스를 나타내는 컬럼이라고 가정
X = df.drop('quality', axis=1).values
y = df['quality'].values

# 클래스 불균형 확인
print('Before Resampling:', Counter(y))

# 클래스별 샘플 수 계산
quality_counts = df['quality'].value_counts().to_dict()
minority_class_count = min(quality_counts.values())

# 오버샘플링
X_over = []
y_over = []
for label, count in quality_counts.items():
    X_label = X[y == label]
    X_label_over = resample(X_label, replace=True, n_samples=count, random_state=42)
    X_over.append(X_label_over)
    y_over.extend([label] * count)

X_over = np.concatenate(X_over)
y_over = np.array(y_over)

# 클래스 불균형 확인
print('After OverSampling:', Counter(y_over))

# 언더샘플링
X_under = []
y_under = []
for label, count in quality_counts.items():
    X_label = X_over[y_over == label]
    X_label_under = resample(X_label, replace=False, n_samples=minority_class_count, random_state=42)
    X_under.append(X_label_under)
    y_under.extend([label] * minority_class_count)

X_under = np.concatenate(X_under)
y_under = np.array(y_under)

# 클래스 불균형 확인
print('After UnderSampling:', Counter(y_under))

Before Resampling: Counter({5: 681, 6: 638, 7: 199, 4: 53, 8: 18, 3: 10})
After OverSampling: Counter({5: 681, 6: 638, 7: 199, 4: 53, 8: 18, 3: 10})
After UnderSampling: Counter({5: 10, 6: 10, 7: 10, 4: 10, 8: 10, 3: 10})


#### 쌍체 t 검정에서는 범주별로 데이터 샘플의 개수가 같아야해서 Undersmapling을 통해 얻은 값들을 사용하였음.

In [9]:
# 데이터에서 품질 클래스에 대한 카운트 가져오기
quality_counts = under_df[under_df.columns[-1]].value_counts()

# 종속 변수와 독립 변수의 컬럼명 설정
dependent_variable = under_df.columns[-1]  # 종속 변수의 컬럼명
independent_variables = under_df.columns[:-1]  # 독립 변수의 컬럼명들

# 각 품질 범주에 대해 쌍체 t-검정 수행
for label1 in quality_counts.keys():
    for label2 in quality_counts.keys():
        if label1 != label2:
            samples_label1 = under_df[under_df[dependent_variable] == label1][independent_variables]
            samples_label2 = under_df[under_df[dependent_variable] == label2][independent_variables]

            # 쌍체 t-검정 수행
            t_statistic, p_value = ttest_rel(samples_label1, samples_label2)

            # 결과 출력
            print(f'T-Test between Class {label1} and Class {label2}:')
            print(f'T-Statistic: {t_statistic}')
            print(f'P-Value: {p_value}')
            print(f'Reject Null Hypothesis: {p_value < 0.05}')
            print('\n')

T-Test between Class 5 and Class 6:
T-Statistic: [-0.71287624  1.00977969 -1.89294677  1.94177081 -1.40749463  1.65091743
  0.62530351  1.29193226  0.57832348 -1.35224194 -1.94597953]
P-Value: [0.49398948 0.33897664 0.09090723 0.08407008 0.19286722 0.13315117
 0.54729752 0.22856775 0.57722836 0.20929526 0.08350405]
Reject Null Hypothesis: [False False False False False False False False False False False]


T-Test between Class 5 and Class 7:
T-Statistic: [-4.45216265  4.48502472 -4.52357125  0.26625432  1.67297436  2.45027491
  2.23945265  1.10549754  2.07363386 -3.15734835 -5.98365797]
P-Value: [1.59520515e-03 1.52137392e-03 1.43940198e-03 7.96042766e-01
 1.28661087e-01 3.67401455e-02 5.18896251e-02 2.97618815e-01
 6.79627407e-02 1.15995892e-02 2.06602302e-04]
Reject Null Hypothesis: [ True  True  True False False  True False False False  True  True]


T-Test between Class 5 and Class 4:
T-Statistic: [ 0.50676024 -1.31581972 -0.52402032 -0.63683164 -0.72007559  2.23236613
  0.3324064

In [8]:
import pandas as pd

# 언더샘플링한 데이터를 데이터프레임으로 변환
under_df = pd.DataFrame(data=X_under, columns=df.columns[:-1])  # 가정: 마지막 열이 타겟 변수
under_df['quality'] = y_under  # 타겟 변수 추가

# T-test 결과

#### Quality 5와 Quality 6 간의 T-Test 결과는 다음과 같습니다:

T-Statistic (t-통계량): 평균 차이를 표현하는 값으로, 음수일 경우 Quality 5가 평균적으로 Quality 6보다 낮음을 나타냅니다.
해당 결과에서는 대부분의 경우 음수이지만 큰 차이가 나타나지 않습니다.

P-Value (유의확률):관찰된 통계량(T-Statistic)을 기준으로 얻은 p-value로, 일반적으로 0.05 미만이면 통계적으로 유의미한 차이가 있다고 판단합니다. 해당 결과에서는 대부분의 경우 0.05보다 크기 때문에 유의미한 차이가 없을 가능성이 높습니다.

Reject Null Hypothesis (귀무가설 기각 여부): 해당 결과에서는 모든 경우에 대해 귀무가설을 기각할 수 있는 충분한 증거가 없다고 판단됩니다. 종합하면, Quality 5와 Quality 6 간에는 통계적으로 유의미한 차이가 나타나지 않을 가능성이 높습니다. 다시 말해, 두 그룹 간의 평균적인 차이는 우연에 의한 것일 가능성이 크다는 것을 의미합니다.

#### 통계적으로 유의미한 차이가 있는 경우:

Quality 5 대 Quality 7: p-값이 매우 작아(0.0015), 귀무가설을 기각하며, 통계적으로 두 그룹 간에 차이가 있다.

Quality 5 대 Quality 8: p-값이 매우 작아(0.0024), 귀무가설을 기각하며, 통계적으로 두 그룹 간에 차이가 있다.

Quality 7 대 Quality 3: p-값이 매우 작아(5.38e-06), 귀무가설을 기각하며, 통계적으로 두 그룹 간에 차이가 있다.

Quality 4 대 Quality 7: p-값이 작아(0.0346), 귀무가설을 기각하며, 통계적으로 두 그룹 간에 차이가 있다.

Quality 8 대 Quality 5: p-값이 매우 작아(0.0024), 귀무가설을 기각하며, 통계적으로 두 그룹 간에 차이가 있다.

#### 통계적으로 유의미한 차이가 없는 경우:

Quality 5 대 Quality 6: p-값이 모두 0.05보다 크기 때문에 귀무가설을 기각할 충분한 증거가 없어, 통계적으로 두 그룹 간에 차이가 없다고 할 수 있습니다.

Quality 5 대 Quality 4: p-값이 모두 0.05보다 크기 때문에 귀무가설을 기각할 충분한 증거가 없어, 통계적으로 두 그룹 간에 차이가 없다고 할 수 있습니다.

Quality 5 대 Quality 3: p-값이 모두 0.05보다 크기 때문에 귀무가설을 기각할 충분한 증거가 없어, 통계적으로 두 그룹 간에 차이가 없다고 할 수 있습니다.

Quality 6 대 Quality 4: p-값이 모두 0.05보다 크기 때문에 귀무가설을 기각할 충분한 증거가 없어, 통계적으로 두 그룹 간에 차이가 없다고 할 수 있습니다.

Quality 6 대 Quality 8: p-값이 모두 0.05보다 크기 때문에 귀무가설을 기각할 충분한 증거가 없어, 통계적으로 두 그룹 간에 차이가 없다고 할 수 있습니다